In [1]:
import os
from dotenv import load_dotenv
import pandas as pd

pd.set_option('display.max_columns', None)
load_dotenv()

endpoint   = os.getenv("MINIO_ENDPOINT")
bucket     = os.getenv("MINIO_BUCKET")
parquet    = os.getenv("MINIO_PARQUET_PATH")
access_key = os.getenv("MINIO_ACCESS_KEY")
secret_key = os.getenv("MINIO_SECRET_KEY")

storage_opts = {
    "key": access_key,
    "secret": secret_key,
    "client_kwargs": {"endpoint_url": endpoint},
}

# Exemplo: abrir o parquet de 2025
path = f"s3://{bucket}/{parquet}/add_producao_2025.parquet"
df = pd.read_parquet(path, storage_options=storage_opts)

In [2]:
df.columns

Index(['an_base', 'id_add_foto_programa', 'id_add_foto_programa_ies',
       'cd_programa', 'nm_programa', 'sg_ies', 'nm_ies', 'cd_entidade_capes',
       'cd_ies_emec', 'id_add_producao_intelectual', 'id_producao_intelectual',
       'nm_producao', 'id_tipo_producao', 'nm_tipo_producao',
       'id_subtipo_producao', 'nm_subtipo_producao', 'id_formulario_producao',
       'nm_formulario', 'id_add_contexto', 'id_area_concentracao',
       'nm_area_concentracao', 'id_linha_pesquisa', 'nm_linha_pesquisa',
       'id_projeto', 'nm_projeto', 'dh_inicio_area_conc', 'dh_fim_area_conc',
       'dh_inicio_linha', 'dh_fim_linha', 'in_glosa', 'sg_estrato',
       'nm_evento_classificacao', 'cd_identificador_veiculo',
       'ds_titulo_padronizado', 'in_periodico', 'id_valor_lista'],
      dtype='object')

In [3]:
df

,an_base,id_add_foto_programa,id_add_foto_programa_ies,cd_programa,nm_programa,sg_ies,nm_ies,cd_entidade_capes,cd_ies_emec,id_add_producao_intelectual,id_producao_intelectual,nm_producao,id_tipo_producao,nm_tipo_producao,id_subtipo_producao,nm_subtipo_producao,id_formulario_producao,nm_formulario,id_add_contexto,id_area_concentracao,nm_area_concentracao,id_linha_pesquisa,nm_linha_pesquisa,id_projeto,nm_projeto,dh_inicio_area_conc,dh_fim_area_conc,dh_inicio_linha,dh_fim_linha,in_glosa,sg_estrato,nm_evento_classificacao,cd_identificador_veiculo,ds_titulo_padronizado,in_periodico,id_valor_lista


In [4]:
# Verificar informações sobre o DataFrame
print(f"Número de linhas: {len(df)}")
print(f"Número de colunas: {len(df.columns)}")
print(f"Forma (shape): {df.shape}")
print(f"DataFrame vazio: {df.empty}")

Número de linhas: 0
Número de colunas: 36
Forma (shape): (0, 36)
DataFrame vazio: True


In [5]:
# Vamos verificar que arquivos existem no bucket MinIO
import s3fs

# Conectar ao MinIO usando s3fs
fs = s3fs.S3FileSystem(
    key=access_key,
    secret=secret_key,
    client_kwargs={"endpoint_url": endpoint}
)

# Listar arquivos no path do parquet
parquet_files = fs.ls(f"{bucket}/{parquet}")
print("Arquivos disponíveis no bucket:")
for file in parquet_files:
    file_name = file.split('/')[-1]
    if 'producao' in file_name.lower():
        print(f"  📄 {file_name}")
    elif 'docente' in file_name.lower():
        print(f"  👨‍🏫 {file_name}")
    else:
        print(f"  📁 {file_name}")

Arquivos disponíveis no bucket:
  👨‍🏫 add_docentes.parquet
  📄 add_producao_2013.parquet
  📄 add_producao_2014.parquet
  📄 add_producao_2015.parquet
  📄 add_producao_2016.parquet
  📄 add_producao_2017.parquet
  📄 add_producao_2018.parquet
  📄 add_producao_2019.parquet
  📄 add_producao_2020.parquet
  📄 add_producao_2021.parquet
  📄 add_producao_2022.parquet
  📄 add_producao_2023.parquet
  📄 add_producao_2024.parquet
  📄 add_producao_2025.parquet


In [6]:
# Vamos testar um arquivo de 2024 para comparar
path_2024 = f"s3://{bucket}/{parquet}/add_producao_2024.parquet"
df_2024 = pd.read_parquet(path_2024, storage_options=storage_opts)

print("Dados de 2024:")
print(f"Número de linhas: {len(df_2024)}")
print(f"Número de colunas: {len(df_2024.columns)}")
print(f"DataFrame vazio: {df_2024.empty}")

if not df_2024.empty:
    print(f"\nPrimeiras 3 linhas:")
    print(df_2024.head(3))

Dados de 2024:
Número de linhas: 1164454
Número de colunas: 36
DataFrame vazio: False

Primeiras 3 linhas:
   an_base  id_add_foto_programa  id_add_foto_programa_ies    cd_programa  \
0     2024                193860                    257893  33002045013P0   
1     2024                194721                    258825  24001015041P6   
2     2024                196238                    260787  33002010182P0   

                 nm_programa            sg_ies  \
0  ENGENHARIA DE TRANSPORTES    USP-SÃO CARLOS   
1       CIÊNCIAS DA NUTRIÇÃO  UFPB-JOÃO PESSOA   
2   CIÊNCIAS DA REABILITAÇÃO               USP   

                                              nm_ies cd_entidade_capes  \
0      UNIVERSIDADE DE SÃO PAULO - CAMPUS SÃO CARLOS          33002045   
1  UNIVERSIDADE FEDERAL DA PARAÍBA - CAMPUS JOÃO ...          24001015   
2                          UNIVERSIDADE DE SÃO PAULO          33002010   

  cd_ies_emec  id_add_producao_intelectual  id_producao_intelectual  \
0        None  

In [7]:
# Vamos verificar o tamanho dos arquivos de produção
anos_teste = ['2023', '2024', '2025']

for ano in anos_teste:
    try:
        path_ano = f"s3://{bucket}/{parquet}/add_producao_{ano}.parquet"
        df_ano = pd.read_parquet(path_ano, storage_options=storage_opts)
        
        # Obter tamanho do arquivo
        info = fs.info(f"{bucket}/{parquet}/add_producao_{ano}.parquet")
        tamanho_mb = info['size'] / (1024 * 1024)
        
        print(f"Ano {ano}:")
        print(f"  📊 Linhas: {len(df_ano):,}")
        print(f"  💾 Tamanho: {tamanho_mb:.2f} MB")
        print(f"  ❓ Vazio: {'Sim' if df_ano.empty else 'Não'}")
        print()
        
    except Exception as e:
        print(f"Erro ao ler {ano}: {e}")
        print()

Ano 2023:
  📊 Linhas: 1,085,440
  💾 Tamanho: 147.01 MB
  ❓ Vazio: Não

Ano 2024:
  📊 Linhas: 1,164,454
  💾 Tamanho: 161.20 MB
  ❓ Vazio: Não

Ano 2025:
  📊 Linhas: 0
  💾 Tamanho: 0.02 MB
  ❓ Vazio: Sim



In [ ]:
# COMPARAÇÃO: add_docentes vs add_producao para dados de IES
print("🏛️ ANÁLISE COMPARATIVA: DADOS DE IES\n")

# 1. Carregar add_docentes.parquet
print("📚 Carregando add_docentes.parquet...")
path_docentes = f"s3://{bucket}/{parquet}/add_docentes.parquet"
df_docentes = pd.read_parquet(path_docentes, storage_options=storage_opts)

# 2. Usar add_producao_2024.parquet (já carregado como df_2024)
print("📊 Usando add_producao_2024.parquet...")

print("=" * 60)
print("📈 ESTATÍSTICAS GERAIS")
print("=" * 60)
print(f"📚 add_docentes   : {len(df_docentes):,} registros")
print(f"📊 add_producao_2024 : {len(df_2024):,} registros")
print()